In [52]:
#load path
import os
import sys
import astropy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.wcs import WCS
from astropy.visualization import (ImageNormalize, ZScaleInterval, LogStretch)
import glob
from mypython.filters import filter as flt

In [53]:
#now do ap phot
from photutils.aperture import CircularAperture, aperture_photometry, CircularAnnulus
from photutils.centroids import centroid_com

reduxpath='./data/Redux/'
bd28names = glob.glob(reduxpath+'/*_NGC6946*.fits')

zpfilter={'R':-16.26,'G':-16.26,'I':-16.06}

skyall = {}
for filt in zpfilter.keys():
    skyall[filt] = []

In [54]:
#write a function to evaluate the median sky in a box at given position

def median_sky_in_box(imagename, x, y, box_size):
    """
    Evaluate the median sky value in a square box centered at (x, y).

    Parameters:
        image (2D np.ndarray): The image array.
        x (int): X-coordinate of the box center.
        y (int): Y-coordinate of the box center.
        box_size (int): Size of the box (length of one side, in pixels).

    Returns:
        float: Median value in the box.
    """

    hdu = fits.open(imagename)[0]
    image = hdu.data
    filt = hdu.header['FILTER']
    scale = np.sqrt(hdu.header['CD1_1']**2 + hdu.header['CD2_1']**2)*3600

    half_size = box_size // 2
    x_min = max(x - half_size, 0)
    x_max = min(x + half_size + 1, image.shape[1])
    y_min = max(y - half_size, 0)
    y_max = min(y + half_size + 1, image.shape[0])
    box = image[y_min:y_max, x_min:x_max]
    return np.median(box), filt, scale



In [55]:
for imgname in bd28names:
    skybrightness, filt, scale = median_sky_in_box(imgname,1288,1792,40)
    skyall[filt].append(skybrightness/scale**2)
    print(scale)


0.5528002497739486
0.5525955860359313
0.5525526304696197
0.5527959102631729
0.5527843687282084
0.5528319516305963
0.5525441060189824
0.552563400067065
0.5525892692392309
0.5528070936280026


In [56]:
print(skyall)

{'R': [9.836590798505972, 9.964423247763577, 9.898290835030325, 10.164562251251159, 10.076451944004335], 'G': [14.775256388563296, 14.51712743829779, 15.4601120110954], 'I': [2.484044645472711, 2.463021043395864]}


In [57]:
#now calibrate the sky
for filt in zpfilter.keys():
    if(len(skyall[filt])>0):
        print('Sky for filter ', filt, ' is ', np.median(skyall[filt])*10**(zpfilter[filt]), ' with std ', np.std(skyall[filt])*10**(zpfilter[filt]), ' from ', len(skyall[filt]), ' measurements')

Sky for filter  R  is  5.475857859063206e-16  with std  6.526653296510204e-18  from  5  measurements
Sky for filter  G  is  8.119607307241494e-16  with std  2.1865909058985506e-17  from  3  measurements
Sky for filter  I  is  2.1543570460627687e-16  with std  9.155395969329441e-19  from  2  measurements


In [ ]:
#enumerate an array

